In [ ]:
import pandas as pd

path = "./../../data/train/"

# Ml_item2attributes.json  directors.tsv  genres.tsv  titles.tsv  train_ratings.csv  writers.tsv  years.tsv

# Load data
titles = pd.read_csv(path + "titles.tsv", sep="\t")
years = pd.read_csv(path + "years.tsv", sep="\t")
genres = pd.read_csv(path + "genres.tsv", sep="\t")
directors = pd.read_csv(path + "directors.tsv", sep="\t")
writers = pd.read_csv(path + "writers.tsv", sep="\t")


ratings = pd.read_csv(path + "train_ratings.csv")

# Merge data

movie_df = pd.merge(titles, years, on="item", how="left")
movie_df = pd.merge(movie_df, genres, on="item", how="left")
movie_df = pd.merge(movie_df, directors, on="item", how="left")
movie_df = pd.merge(movie_df, writers, on="item", how="left")

# Group by 'item' and aggregate genres and writers
movie_df = movie_df.groupby(['item', 'title', 'year', 'director']).agg({
    'genre': lambda x: ', '.join(x.unique()),
    'writer': lambda x: ', '.join(x.unique())
}).reset_index()

print(movie_df.head())


   item                             title    year   genre   director  \
0   318  Shawshank Redemption, The (1994)  1994.0   Crime  nm0001104   
1   318  Shawshank Redemption, The (1994)  1994.0   Crime  nm0001104   
2   318  Shawshank Redemption, The (1994)  1994.0   Drama  nm0001104   
3   318  Shawshank Redemption, The (1994)  1994.0   Drama  nm0001104   
4  2571                Matrix, The (1999)  1999.0  Action  nm0905152   

      writer  
0  nm0000175  
1  nm0001104  
2  nm0000175  
3  nm0001104  
4  nm0905152  


In [ ]:
""